In [19]:
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

In [24]:
from langchain_openai import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

paul_graham_essays = './startupideas.txt'

with open(paul_graham_essays, 'r') as file:
    essay = file.read()

In [25]:
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
llm.get_num_tokens(essay)

9336

In [27]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=10000, chunk_overlap=500)

docs = text_splitter.create_documents([essay])

In [28]:
num_docs = len(docs)

num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

print (f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens")

Now we have 5 documents and the first one has 2043 tokens


In [29]:
summary_chain = load_summarize_chain(llm=llm, chain_type='map_reduce',
#                                      verbose=True # Set verbose=True if you want to see the prompts being used
                                    )

In [30]:
output = summary_chain.run(docs)

/Users/huke/openai-learning/venv2405/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [32]:
output

'\n\nY Combinator is a funding source for startups that emphasizes the importance of identifying and solving problems that are personally relevant. It advises against creating solutions for non-existent problems and suggests living in the future and being at the forefront of a rapidly changing field to generate good startup ideas. The essay also discusses the importance of gaining experience and turning off filters to come up with successful ideas. It emphasizes the need for a strong motivation and targeting niche markets to create a successful startup.'

In [33]:
map_prompt = """
Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:
"""
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])

In [34]:
combine_prompt = """
Write a concise summary of the following text delimited by triple backquotes.
Return your response in bullet points which covers the key points of the text.
```{text}```
BULLET POINT SUMMARY:
"""
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text"])

In [35]:
summary_chain = load_summarize_chain(llm=llm,
                                     chain_type='map_reduce',
                                     map_prompt=map_prompt_template,
                                     combine_prompt=combine_prompt_template,
#                                      verbose=True
                                    )

In [36]:
output = summary_chain.run(docs)

In [37]:
print (output)

- Y Combinator advises entrepreneurs to solve problems they have personally experienced
- Finding a niche market with strong demand is important
- Being at the forefront of a rapidly changing field and having strong intuition are key factors in generating good startup ideas
- Living in the future and noticing what is missing in the world is crucial for success
- Taking a long-term view and focusing on building what is interesting is important
- Building things and gaining experience is more valuable than taking entrepreneurship classes
- Turning off filters, such as fear of tedious tasks and aversion to unsexy ideas, is necessary for success
- The organic method is emphasized for coming up with startup ideas
- Being young and having expertise in a certain area can be beneficial
- Looking for unmet needs and solving problems for others is recommended
- College can help students start startups
- Focusing on users rather than competitors is important
- Tips for finding successful startup 